In [2]:
import pymssql
import pandas
import yaml

import sys
from pathlib import Path
sys.path.append('C:\\Users\\ysman\\Desktop\\Capstone\\PIVOT\\PIVOT')


In [18]:
import utils

In [22]:
from importlib import reload

In [3]:
import utils.sql_utils as sq

In [9]:
sys.path.append(parent_dir)

In [4]:
%load_ext autoreload
%autoreload 2

In [11]:
from importlib import reload

In [89]:
import sql_utils, sql_constants
reload(sql_utils)
reload(sql_constants)

<module 'sql_constants' from 'C:\\Users\\ysman\\Desktop\\Capstone\\PIVOT\\sql_constants.py'>

In [78]:
df = sql_utils.get_label_rank_df(model_id=1,dissimilarity_id=1,batch_size=5,)

In [6]:
sq.get_test_set_df(model_id=1,minimum_percent=0.0, sp_name='MODEL_EVALUATION_NON_TEST')

,IMAGE_ID,PRED_LABEL,CONSENSUS
0,3,Confirmed Label A,Confirmed Label A


In [ ]:
sql_utils.get_test_set_df(model_id=1,minimum_percent=0.0)

In [102]:
sql_utils.run_sql_query(
    """
    Insert into labels (i_id, u_id, weight, date, label)
    Values (3,2, 1, '2024-01-27 12:00:00', 'Confirmed Label A' );
    """)

IntegrityError: (547, b'The INSERT statement conflicted with the FOREIGN KEY constraint "FK__labels__i_id__123EB7A3". The conflict occurred in database "capstoneazure", table "dbo.images", column \'i_id\'.DB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\n')

In [79]:
df

,IMAGE_ID,BLOB_FILEPATH,UNCERTAINTY,PRED_LABEL,PROBS,RANK_SCORE
0,2,path/to/image2.jpg,0.6,Label B,0.8,0.455288
1,1,path/to/image1.jpg,0.5,Label A,0.9,0.435549
0,1,path/to/image1.jpg,1.0,Label A,0.9,0.871099
1,2,path/to/image2.jpg,1.0,Label B,0.8,0.758813


In [31]:
sq.run_sql_query("select * from labels;")

KeyError: 'server'

In [97]:
sql_utils.run_sql_query(
"""
SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE, COLUMN_DEFAULT
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'predictions';
"""
)

,COLUMN_NAME,DATA_TYPE,IS_NULLABLE,COLUMN_DEFAULT
0,m_id,int,NO,None
1,i_id,int,NO,None
2,class_prob,varchar,YES,None
3,pred_label,varchar,YES,None


In [76]:
sql_utils.run_sql_query("""
UPDATE labels
SET weight = weight / 10;
""")

C:\Users\ysman\AppData\Local\Temp\ipykernel_11480\1445388442.py:1: UserWarning: The query returned empty.
  sql_utils.run_sql_query("""


In [98]:
sql_utils.run_sql_query(
    """
    Insert into predictions (m_id, i_id, class_prob, pred_label)
    Values (1,4, '0.5', 'Confirmed Label A' );
    """)

C:\Users\ysman\AppData\Local\Temp\ipykernel_11480\3509736632.py:1: UserWarning: The query returned empty.
  sql_utils.run_sql_query(


In [99]:
sql_utils.run_sql_query("select * from predictions;")

,m_id,i_id,class_prob,pred_label
0,1,1,0.9,Label A
1,1,2,0.8,Label B
2,1,3,0.5,Confirmed Label A
3,2,1,0.7,Label C
4,2,2,0.6,Label D


In [103]:
sql_utils.run_sql_query("delete from metrics where i_id = 3 and d_id=0;")

C:\Users\ysman\AppData\Local\Temp\ipykernel_11480\2975923993.py:1: UserWarning: The query returned empty.
  sql_utils.run_sql_query("delete from metrics where i_id = 3 and d_id=0;")


In [104]:
sql_utils.run_sql_query("select * from metrics;")

,i_id,m_id,d_id,d_value
0,1,0,0,1.0
1,2,0,0,1.0
2,6,0,0,1.0
3,7,0,0,1.0
4,1,1,1,0.5
5,2,1,1,0.6
6,1,2,2,0.4
7,2,2,2,0.3


In [48]:
sql_utils.run_sql_query("select * from dissimilarity;")

,d_id,name,formula
0,0,random_sample,none
1,1,Dissimilarity Metric A,Formula A
2,2,Dissimilarity Metric B,Formula B


In [63]:
sql_utils.run_sql_query("select * from predictions;")

,m_id,i_id,class_prob,pred_label
0,1,1,0.9,Label A
1,1,2,0.8,Label B
2,2,1,0.7,Label C
3,2,2,0.6,Label D


In [47]:
sql_utils.run_sql_query(
"""
SET IDENTITY_INSERT dissimilarity ON;


INSERT INTO dissimilarity (d_id, name, formula)
VALUES (0, 'random_sample', 'none');
SET IDENTITY_INSERT dissimilarity OFF;
"""
)

In [45]:
try:
    print(hi)
except Exception as e:
    print(e.

<class 'NameError'>


In [71]:
a  = sql_utils.run_sql_query("""
WITH LABEL_COUNTS AS (
        SELECT
            I_ID,
            SUM(WEIGHT) AS W_COUNT
        FROM LABELS
        GROUP BY I_ID
    )
    SELECT
           I.I_ID AS IMAGE_ID,
           I.FILEPATH AS BLOB_FILEPATH,
           M.D_VALUE AS UNCERTAINTY,
           P.PRED_LABEL AS PRED_LABEL,
           P.CLASS_PROB AS PROBS,
           P.M_ID,
           M.M_ID,
           M.D_ID
          -- IIF(L.W_COUNT < 10, (M.D_VALUE * EXP(-0.069 * L.W_COUNT)), 0) AS RANK_SCORE
    FROM METRICS AS M
    INNER JOIN IMAGES AS I
        ON M.I_ID = I.I_ID
    INNER JOIN PREDICTIONS AS P
        ON M.I_ID = P.I_ID AND (M.M_ID = P.M_ID OR M.M_ID = 0) -- allow for test images that have M_ID=0
    INNER JOIN LABEL_COUNTS AS L
        ON M.I_ID = L.I_ID
    WHERE
          P.M_ID = 1
      AND M.D_ID = 0
    --ORDER BY RANK_SCORE DESC;
""")

check_fk = """
SELECT
    fk.name AS ForeignKeyName,
    tp.name AS ReferencedTable,
    ref_cols.name AS ReferencedColumn,
    tab.name AS TableName,
    fk_cols.name AS ColumnName
FROM
    sys.foreign_keys AS fk
JOIN
    sys.tables AS tab ON fk.parent_object_id = tab.object_id
JOIN
    sys.tables AS tp ON fk.referenced_object_id = tp.object_id
JOIN
    sys.foreign_key_columns AS fkc ON fkc.constraint_object_id = fk.object_id
JOIN
    sys.columns AS fk_cols ON fk_cols.object_id = fkc.parent_object_id AND fk_cols.column_id = fkc.parent_column_id
JOIN
    sys.columns AS ref_cols ON ref_cols.object_id = fkc.referenced_object_id AND ref_cols.column_id = fkc.referenced_column_id
WHERE
    fk.name = 'FK__metrics__m_id__0F624AF8';
"""
a

,IMAGE_ID,BLOB_FILEPATH,UNCERTAINTY,PRED_LABEL,PROBS,M_ID,M_ID,D_ID
0,1,path/to/image1.jpg,1.0,Label A,0.9,1,0,0
1,2,path/to/image2.jpg,1.0,Label B,0.8,1,0,0


In [62]:
a

,IMAGE_ID,BLOB_FILEPATH,UNCERTAINTY,PRED_LABEL,PROBS,RANK_SCORE
0,1,path/to/image1.jpg,1.0,Label C,0.7,0.0
1,2,path/to/image2.jpg,1.0,Label B,0.8,0.0


In [52]:
sql_utils.generate_random_evaluation_set(5)

In [111]:
sql_utils.execute_stored_procedure("AL_RANKINGS", args=args)

shit


,IMAGE_ID,BLOB_FILEPATH,UNCERTAINTY,PRED_LABEL,PROBS,RANK_SCORE
0,1,path/to/image1.jpg,0.5,Label A,0.9,0.0
1,2,path/to/image2.jpg,0.6,Label B,0.8,0.0


In [114]:
if 1<= 2<=3:
    print("hi")

hi


In [15]:
sql_utils.create_alter_stored_procedure('GENERATE_RANDOM_TEST_SET')

Using preset file to create procedure GENERATE_RANDOM_TEST_SET: C:\Users\ysman\Desktop\Capstone\PIVOT\stored_procedures\Generate_Random_Test_Set.sql


In [ ]:
sql_utils.create_alter_stored_procedure('AL_RANKINGS')

In [80]:
sql_utils.create_alter_stored_procedure('MODEL_EVALUATION_MAX_CONSENSUS_FILTERING')

Using preset file to create procedure MODEL_EVALUATION_MAX_CONSENSUS_FILTERING: C:\Users\ysman\Desktop\Capstone\PIVOT\stored_procedures\Model_Evaluation_Filtering.sql


In [88]:
sql_utils.create_alter_stored_procedure('MODEL_EVALUATION_NON_TEST')

Using preset file to create procedure MODEL_EVALUATION_NON_TEST: C:\Users\ysman\Desktop\Capstone\PIVOT\stored_procedures\Model_Evaluation_NonTest.sql


In [63]:
from collections import OrderedDict
args = OrderedDict([
    ("M_ID", 1),
    ("D_ID", 1),
    ("RELABEL_LAMBDA", 0.069),
    ("BATCH_SIZE", 3),
])

In [74]:
(if df) and (if df):

SyntaxError: invalid syntax (1904830851.py, line 1)

In [70]:
pd.concat([df, df],)

,IMAGE_ID,BLOB_FILEPATH,UNCERTAINTY,PRED_LABEL,PROBS,RANK_SCORE
0,2,path/to/image2.jpg,0.6,Label B,0.8,0.0
1,1,path/to/image1.jpg,0.5,Label A,0.9,0.0
0,2,path/to/image2.jpg,0.6,Label B,0.8,0.0
1,1,path/to/image1.jpg,0.5,Label A,0.9,0.0


In [97]:
assert d

In [92]:
df

,IMAGE_ID,BLOB_FILEPATH,UNCERTAINTY,PRED_LABEL,PROBS,RANK_SCORE
0,1,path/to/image1.jpg,0.5,Label A,0.9,0.0
1,2,path/to/image2.jpg,0.6,Label B,0.8,0.0


In [41]:
check_procedures = """
    SELECT 
        name AS 'ProcedureName',
        create_date AS 'CreateDate',
        modify_date AS 'ModifyDate'
    FROM 
        sys.procedures;
"""

In [95]:
import pymssql

server = 'capstoneservercjault.database.windows.net'
user = 'CloudSA85da88d3'
password = 'Passwordtest123'
database = 'capstoneazure'


with pymssql.connect(server, user, password, database) as conn:
    with conn.cursor() as cursor:
        cursor.execute("""
        select * from Labels;
        """)
#         row = cursor.fetchone()
        row = cursor.fetchall()
#         while row:
#             print("SQL Server version:", row[0])
#             row = cursor.fetchone()
        columns = [column[0] for column in cursor.description]
        print(cursor.description)
df = pd.DataFrame(row, columns=columns)
df.head()

(('i_id', 3, None, None, None, None, None), ('u_id', 3, None, None, None, None, None), ('weight', 3, None, None, None, None, None), ('date', 4, None, None, None, None, None), ('label', 1, None, None, None, None, None))


,i_id,u_id,weight,date,label
0,1,1,10,2024-01-26 10:00:00,Confirmed Label A
1,2,1,20,2024-01-26 11:00:00,Confirmed Label B
2,1,2,15,2024-01-26 12:00:00,Confirmed Label C
3,2,2,25,2024-01-26 13:00:00,Confirmed Label D


In [27]:
df

,m_id,model_name,model_link,structure,weights
0,1,Model A,http://link-to-modela.com,JSON structure A,Binary data A
1,2,Model B,http://link-to-modelb.com,JSON structure B,Binary data B
